# Assistants API

## 💡 这节课会带给你

1. 原生 API、GPTs 和 Assistants API 的适用场景
2. 用 Assistants API 做一个 GPT

开始上课！


## 🎓 这节课怎么学

代码能力要求：**中低**，AI/数学基础要求：**无**

1. 有编程基础的同学
   - 关注代码实现细节、应用场景
2. 没有编程基础的同学
   - 关注 OpenAI 提供的能力和产品形态，多思考为什么/有什么优缺点


## 前言

### 0.1、从轰动一时的 OpenAI DevDay 说起

2023 年 11 月 6 日，OpenAI DevDay 发表了一系列新能力，其中包括：**GPT Store** 和 **Assistants API**

<img src="dawn_of_gpts.jpg" width=600px>

这一波操作一度被认为是创业公司终结者

<img src="post.jpg" width=600px>

几天后更戏剧性一幕就不在课上展开了...

<img src="ouster.png" width=600px>


### 0.2、GPTs 和 Assistants API 本质是降低开发门槛

可操控性和易用性之间的权衡与折中：

1. 更多技术路线选择：原生 API、GPTs 和 Assistants API
2. GPTs 的示范，起到教育客户的作用，有助于打开市场
3. 要更大自由度，需要用 Assistants API 开发
4. 想极致调优，还得原生 API + RAG
5. 国内大模型的 Assistants API，参考 [Minimax](https://www.minimaxi.com/document/guides/Assistants/operate?id=6586b8674da4834fd75906e7)


### 0.3、Assistants API 的主要能力

已有能力：

1. 创建和管理 assistant，每个 assistant 有独立的配置
2. 支持无限长的多轮对话，对话历史保存在 OpenAI 的服务器上
3. 通过自有向量数据库支持基于文件的 RAG
4. 支持 Code Interpreter
   1. 在沙箱里编写并运行 Python 代码
   2. 自我修正代码
   3. 可传文件给 Code Interpreter
5. 支持 Function Calling
6. 支持在线调试的 Playground

承诺未来会有的能力：

1. 支持 DALL·E
2. 支持图片消息
3. 支持自定义调整 RAG 的配置项

收费：

1. 按 token 收费。无论多轮对话，还是 RAG，所有都按实际消耗的 token 收费
2. 如果对话历史过多超过大模型上下文窗口，会自动放弃最老的对话消息
3. 文件按数据大小和存放时长收费。1 GB **向量存储** 一天收费 0.10 美元
4. Code interpreter 跑一次 $0.03


## 一、GPT Store：创建自己的 GPT

<img src="create_gpt.png" width=800px />

<img src="gpt.png" width=800px />

发布链接：https://chat.openai.com/g/g-iU8hVr4jR-wo-de-demogpt


## 二、Assistants API


In [34]:
!pip install --upgrade openai

### 2.1、创建一个 Assistant

可以为每个应用，甚至应用中的每个有对话历史的使用场景，创建一个 assistant。

虽然可以用代码创建，也不复杂，例如：

```python
from openai import OpenAI

# 初始化 OpenAI 服务
client = OpenAI()

# 创建助手
assistant = client.beta.assistants.create(
    name="AGIClass Demo",
    instructions="你叫瓜瓜，你是AGI课堂的智能助理。你负责回答与AGI课堂有关的问题。",
    model="gpt-4-turbo",
)
```

但是，更佳做法是，到 [Playground](https://platform.openai.com/playground?mode=assistant) 在线创建，因为：

1. 更方便调整
2. 更方便测试


In [47]:
from openai import OpenAI

# 初始化 OpenAI 服务
client = OpenAI()

# 创建助手
assistant = client.beta.assistants.create(
    name="AGIClass Demo2",
    instructions="你叫瓜瓜，你是AGI课堂的智能助理。你负责回答与AGI课堂有关的问题。",
    model="gpt-4-turbo",
)

print(assistant.id)

asst_D1i0KrDlmfXkuXP1pgVhiguG


### 2.2、样例 Assistant 的配置

Instructions:

```
你叫瓜瓜。你是AGI课堂的助手。你只回答跟AI大模型有关的问题。不要跟学生闲聊。每次回答问题前，你要拆解问题并输出一步一步的思考过程。
```

Functions:

```JSON
{
  "name": "ask_database",
  "description": "Use this function to answer user questions about course schedule. Output should be a fully formed SQL query.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "SQL query extracting info to answer the user's question.\nSQL should be written using this database schema:\n\nCREATE TABLE Courses (\n\tid INT AUTO_INCREMENT PRIMARY KEY,\n\tcourse_date DATE NOT NULL,\n\tstart_time TIME NOT NULL,\n\tend_time TIME NOT NULL,\n\tcourse_name VARCHAR(255) NOT NULL,\n\tinstructor VARCHAR(255) NOT NULL\n);\n\nThe query should be returned in plain text, not in JSON.\nThe query should only contain grammars supported by SQLite."
      }
    },
    "required": [
      "query"
    ]
  }
}
```


上传文件:

[《AI ⼤模型全栈⼯程师培养计划》](./agiclass_intro.pdf)


## 三、代码访问 Assistant


### 3.1、管理 thread

Threads：

1. Threads 里保存的是对话历史，即 messages
2. 一个 assistant 可以有多个 thread
3. 一个 thread 可以有无限条 message
4. 一个用户与 assistant 的多轮对话历史可以维护在一个 thread 里


In [48]:
import json


def show_json(obj):
    """把任意对象用排版美观的 JSON 格式打印出来"""
    print(json.dumps(
        json.loads(obj.model_dump_json()),
        indent=4,
        ensure_ascii=False
    ))

In [49]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 初始化 OpenAI 服务
client = OpenAI()   # openai >= 1.3.0 起，OPENAI_API_KEY 和 OPENAI_BASE_URL 会被默认使用

# 创建 thread
thread = client.beta.threads.create()
show_json(thread)

{
    "id": "thread_brlL53fDzbhhaIppylHnWOFu",
    "created_at": 1714574031,
    "metadata": {},
    "object": "thread"
}


可以根据需要，自定义 `metadata`，比如创建 thread 时，把 thread 归属的用户信息存入。


In [50]:
thread = client.beta.threads.create(
    metadata={"fullname": "王卓然", "username": "taliux"}
)
show_json(thread)

{
    "id": "thread_sb0ElbtD31MusgOoEo9OfjRx",
    "created_at": 1714574032,
    "metadata": {
        "fullname": "王卓然",
        "username": "taliux"
    },
    "object": "thread"
}


Thread ID 如果保存下来，是可以在下次运行时继续对话的。

从 thread ID 获取 thread 对象的代码：


In [51]:
thread = client.beta.threads.retrieve(thread.id)
show_json(thread)

{
    "id": "thread_sb0ElbtD31MusgOoEo9OfjRx",
    "created_at": 1714574032,
    "metadata": {
        "fullname": "王卓然",
        "username": "taliux"
    },
    "object": "thread"
}


此外，还有：

1. `threads.modify()` 修改 thread 的 `metadata` 和 `tool_resources`
2. `threads.retrieve()` 获取 thread
3. `threads.delete()` 删除 thread。

具体文档参考：https://platform.openai.com/docs/api-reference/threads


### 3.2、给 Threads 添加 Messages

这里的 messages 结构要复杂一些：

1.  不仅有文本，还可以有图片和文件
2.  也有 `metadata`


In [52]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # message 必须归属于一个 thread
    role="user",          # 取值是 user 或者 assistant。但 assistant 消息会被自动加入，我们一般不需要自己构造
    content="你都能做什么？",
)
show_json(message)

{
    "id": "msg_gnnuYHzNpylcGvJmNiHtsmdd",
    "assistant_id": null,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "你都能做什么？"
            },
            "type": "text"
        }
    ],
    "created_at": 1714574033,
    "file_ids": [],
    "metadata": {},
    "object": "thread.message",
    "role": "user",
    "run_id": null,
    "thread_id": "thread_sb0ElbtD31MusgOoEo9OfjRx"
}


还有如下函数：

1. `threads.messages.retrieve()` 获取 message
2. `threads.messages.update()` 更新 message 的 `metadata`
3. `threads.messages.list()` 列出给定 thread 下的所有 messages

具体文档参考：https://platform.openai.com/docs/api-reference/messages


也可以在创建 thread 同时初始化一个 message 列表


In [53]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "你好",
        },
        {
            "role": "assistant",
            "content": "有什么可以帮您？",
        },
        {
            "role": "user",
            "content": "你是谁？",
        },
    ]
)

show_json(thread)  # 显示 thread
print("-----")
show_json(client.beta.threads.messages.list(
    thread.id))  # 显示指定 thread 中的 message 列表

{
    "id": "thread_p97htXj3vfCOCumuKMSYrvtP",
    "created_at": 1714574035,
    "metadata": {},
    "object": "thread"
}
-----
{
    "data": [
        {
            "id": "msg_NAulXln93bswLimNYsozx5Z5",
            "assistant_id": null,
            "content": [
                {
                    "text": {
                        "annotations": [],
                        "value": "你是谁？"
                    },
                    "type": "text"
                }
            ],
            "created_at": 1714574035,
            "file_ids": [],
            "metadata": {},
            "object": "thread.message",
            "role": "user",
            "run_id": null,
            "thread_id": "thread_p97htXj3vfCOCumuKMSYrvtP"
        },
        {
            "id": "msg_fSVSXZq1iaCJrrcp8AaFavqM",
            "assistant_id": null,
            "content": [
                {
                    "text": {
                        "annotations": [],
                        "value": "有什么可以帮您？"
 

## 3.3、开始 Run

- 用 run 把 assistant 和 thread 关联，进行对话
- 一个 prompt 就是一次 run


### 3.1、直接运行


In [54]:
assistant_id = "asst_D4hV5Df5LjOnjaSvYMtjMIch"  # 从 Playground 中拷贝

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

AttributeError: 'Runs' object has no attribute 'create_and_poll'

In [44]:
if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    show_json(messages)
else:
    print(run.status)

NameError: name 'run' is not defined

### 3.2、Run 的状态（选）


Run 的底层是个异步调用，意味着它不等大模型处理完，就返回。我们通过 `run.status` 了解大模型的工作进展情况，来判断下一步该干什么。

`run.status` 有的状态，和状态之间的转移关系如图。

<img src="statuses.png" width="800" />


### 3.3、流式运行


1. 创建回调函数


In [2]:
from typing_extensions import override
from openai import AssistantEventHandler


class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        """响应输出创建事件"""
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """响应输出生成的流片段"""
        print(delta.value, end="", flush=True)

ImportError: cannot import name 'AssistantEventHandler' from 'openai' (/opt/conda/lib/python3.11/site-packages/openai/__init__.py)

2. 运行 run


In [ ]:
# 添加新一轮的 user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="你说什么？",
)
# 使用 stream 接口并传入 EventHandler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant_id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

还有如下函数：

1. `threads.runs.list()` 列出 thread 归属的 run
2. `threads.runs.retrieve()` 获取 run
3. `threads.runs.update()` 修改 run 的 metadata
4. `threads.runs.cancel()` 取消 `in_progress` 状态的 run

具体文档参考：https://platform.openai.com/docs/api-reference/runs


<div class="alert alert-warning">
<b>思考：</b> 进一步理解 run 与 thread 的设计
<ul>
    <li>抛开 Assistants API，假设你要开发任意一个多轮对话的 AI 机器人</li>
    <li>从架构设计的角度，应该怎么维护用户、对话历史、对话引擎、对话服务？</li>
</ul>
</div>


## 四、使用 Tools


### 4.1、创建 Assistant 时声明 Code_Interpreter


如果用代码创建：

```python
assistant = client.beta.assistants.create(
    name="Demo Assistant",
    instructions="你是人工智能助手。你可以通过代码回答很多数学问题。",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo"
)
```


在回调中加入 code_interpreter 的事件响应


In [3]:
from typing_extensions import override
from openai import AssistantEventHandler


class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        """响应输出创建事件"""
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """响应输出生成的流片段"""
        print(delta.value, end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        """响应工具调用"""
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_tool_call_delta(self, delta, snapshot):
        """响应工具调用的流片段"""
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
        if delta.code_interpreter.outputs:
            print(f"\n\noutput >", flush=True)
            for output in delta.code_interpreter.outputs:
                if output.type == "logs":
                    print(f"\n{output.logs}", flush=True)

ImportError: cannot import name 'AssistantEventHandler' from 'openai' (/opt/conda/lib/python3.11/site-packages/openai/__init__.py)

发个 Code Interpreter 请求


In [4]:
# 创建 thread
thread = client.beta.threads.create()

# 添加新一轮的 user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="用代码计算 1234567 的平方根",
)
# 使用 stream 接口并传入 EventHandler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant_id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

NameError: name 'client' is not defined

### 4.1.1、Code_Interpreter 操作文件


In [5]:
# 上传文件到 OpenAI
file = client.files.create(
    file=open("mydata.csv", "rb"),
    purpose='assistants'
)

# 创建 assistant
my_assistant = client.beta.assistants.create(
    name="CodeInterpreterWithFileDemo",
    instructions="你是数据分析师，按要求分析数据。",
    model="gpt-4-turbo",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
          "file_ids": [file.id]  # 为 code_interpreter 关联文件
        }
    }
)

NameError: name 'client' is not defined

In [6]:
# 创建 thread
thread = client.beta.threads.create()

# 添加新一轮的 user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="统计总销售额",
)
# 使用 stream 接口并传入 EventHandler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=my_assistant.id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

NameError: name 'client' is not defined

关于文件操作，还有如下函数：

1. `client.files.list()` 列出所有文件
2. `client.files.retrieve()` 获取文件对象
3. `client.files.delete()` 删除文件
4. `client.files.content()` 读取文件内容

具体文档参考：https://platform.openai.com/docs/api-reference/files


### 4.2、创建 Assistant 时声明 Function


如果用代码创建：

```python
assistant = client.beta.assistants.create(
  instructions="你叫瓜瓜。你是AGI课堂的助手。你只回答跟AI大模型有关的问题。不要跟学生闲聊。每次回答问题前，你要拆解问题并输出一步一步的思考过程。",
  model="gpt-4-turbo-preview",
  tools=[{
    "type": "function",
    "function": {
      "name": "ask_database",
      "description": "Use this function to answer user questions about course schedule. Output should be a fully formed SQL query.",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "SQL query extracting info to answer the user's question.\nSQL should be written using this database schema:\n\nCREATE TABLE Courses (\n\tid INT AUTO_INCREMENT PRIMARY KEY,\n\tcourse_date DATE NOT NULL,\n\tstart_time TIME NOT NULL,\n\tend_time TIME NOT NULL,\n\tcourse_name VARCHAR(255) NOT NULL,\n\tinstructor VARCHAR(255) NOT NULL\n);\n\nThe query should be returned in plain text, not in JSON.\nThe query should only contain grammars supported by SQLite."
          }
        },
        "required": [
          "query"
        ]
    }
  }]
)
```


创建一个 Function


In [16]:
# 定义本地函数和数据库

import sqlite3

# 创建数据库连接
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 创建orders表
cursor.execute("""
CREATE TABLE Courses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    course_date DATE NOT NULL,
    start_time TIME NOT NULL,
    end_time TIME NOT NULL,
    course_name VARCHAR(255) NOT NULL,
    instructor VARCHAR(255) NOT NULL
);
""")

# 插入5条明确的模拟记录
timetable = [
    ('2024-01-23', '20:00', '22:00', '大模型应用开发基础', '孙志岗'),
    ('2024-01-25', '20:00', '22:00', 'Prompt Engineering', '孙志岗'),
    ('2024-01-29', '20:00', '22:00', '赠课：软件开发基础概念与环境搭建', '西树'),
    ('2024-02-20', '20:00', '22:00', '从AI编程认知AI', '林晓鑫'),
    ('2024-02-22', '20:00', '22:00', 'Function Calling', '孙志岗'),
    ('2024-02-29', '20:00', '22:00', 'RAG和Embeddings', '王卓然'),
    ('2024-03-05', '20:00', '22:00', 'Assistants API', '王卓然'),
    ('2024-03-07', '20:00', '22:00', 'Semantic Kernel', '王卓然'),
    ('2024-03-14', '20:00', '22:00', 'LangChain', '王卓然'),
    ('2024-03-19', '20:00', '22:00', 'LLM应用开发工具链', '王卓然'),
    ('2024-03-21', '20:00', '22:00', '手撕 AutoGPT', '王卓然'),
    ('2024-03-26', '20:00', '22:00', '模型微调（上）', '王卓然'),
    ('2024-03-28', '20:00', '22:00', '模型微调（下）', '王卓然'),
    ('2024-04-09', '20:00', '22:00', '多模态大模型（上）', '多老师'),
    ('2024-04-11', '20:00', '22:00', '多模态大模型（中）', '多老师'),
    ('2024-04-16', '20:00', '22:00', '多模态大模型（下）', '多老师'),
    ('2024-04-18', '20:00', '22:00', 'AI产品部署和交付（上）', '王树冬'),
    ('2024-04-23', '20:00', '22:00', 'AI产品部署和交付（下）', '王树冬'),
    ('2024-04-25', '20:00', '22:00', '抓住大模型时代的创业机遇', '孙志岗'),
    ('2024-05-07', '20:00', '22:00', '产品运营和业务沟通', '孙志岗'),
    ('2024-05-09', '20:00', '22:00', '产品设计', '孙志岗'),
    ('2024-05-14', '20:00', '22:00', '项目方案分析与设计', '王卓然'),
]

for record in timetable:
    cursor.execute('''
    INSERT INTO Courses (course_date, start_time, end_time, course_name, instructor)
    VALUES (?, ?, ?, ?, ?)
    ''', record)

# 提交事务
conn.commit()


def ask_database(query):
    cursor.execute(query)
    records = cursor.fetchall()
    return str(records)


# 可以被回调的函数放入此字典
available_functions = {
    "ask_database": ask_database,
}

增加回调事件的响应


In [17]:
from typing_extensions import override
from openai import AssistantEventHandler


class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        """响应回复创建事件"""
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """响应输出生成的流片段"""
        print(delta.value, end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        """响应工具调用"""
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_tool_call_delta(self, delta, snapshot):
        """响应工具调用的流片段"""
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

    @override
    def on_event(self, event):
        """
        响应 'requires_action' 事件
        """
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id  # 获取 run ID
            self.handle_requires_action(event.data, run_id)

    def handle_requires_action(self, data, run_id):
        tool_outputs = []

        for tool in data.required_action.submit_tool_outputs.tool_calls:
            arguments = json.loads(tool.function.arguments)
            print(
                f"{tool.function.name}({arguments})",
                flush=True
            )
            # 运行 function
            tool_outputs.append({
                "tool_call_id": tool.id,
                "output": available_functions[tool.function.name](
                    **arguments
                )}
            )

        # 提交 function 的结果，并继续运行 run
        self.submit_tool_outputs(tool_outputs, run_id)

    def submit_tool_outputs(self, tool_outputs, run_id):
        """提交function结果，并继续流"""
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            stream.until_done()

In [18]:
# 创建 thread
thread = client.beta.threads.create()

# 添加 user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="平均一堂课多长时间",
)
# 使用 stream 接口并传入 EventHandler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant_id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > function

ask_database({'query': "SELECT AVG(strftime('%s', end_time) - strftime('%s', start_time)) FROM Courses"})

assistant > 平均一堂课的时间是2小时。

### 4.3、两个无依赖的 function 会在一次请求中一起被调用


In [19]:
# 创建 thread
thread = client.beta.threads.create()

# 添加 user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="王卓然上几堂课，比孙志岗多上几堂",
)
# 使用 stream 接口并传入 EventHandler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant_id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > function


assistant > function

ask_database({'query': "SELECT instructor, COUNT(instructor) as course_count FROM Courses WHERE instructor IN ('王卓然', '孙志岗') GROUP BY instructor;"})
ask_database({'query': "SELECT instructor, COUNT(instructor) as course_count FROM Courses WHERE instructor IN ('王卓然', '孙志岗') GROUP BY instructor;"})

assistant > 王卓然共上了9堂课，而孙志岗上了6堂课。因此，王卓然比孙志岗多上了3堂课。

<div class="alert alert-info">
<b>更多流中的 Event：</b> https://platform.openai.com/docs/api-reference/assistants-streaming/events
</div>


## 五、内置的 RAG 功能


### 5.1、创建 Vector Store，上传文件


- 通过代码创建 Vector Store


```python
vector_store = client.beta.vector_stores.create(
  name="MyVectorStore"
)
```

```

```


- 通过代码上传文件到 OpenAI 的存储空间


```python
file = client.files.create(
  file=open("agiclass_intro.pdf", "rb"),
  purpose="assistants"
)
```


- 通过代码将文件添加到 Vector Store


```python
vector_store_file = client.beta.vector_stores.files.create(
  vector_store_id=vector_store.id,
  file_id=file.id
)
```


- 批量上传文件到 Vector Store


```python
files = ['file1.pdf','file2.pdf']

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=[open(filename, "rb") for filename in files]
)
```


Vector store 和 vector store file 也有对应的 `list`, `retrieve`, 和 `delete` 等操作。

具体文档参考：

- Vector store: https://platform.openai.com/docs/api-reference/vector-stores
- Vector store file: https://platform.openai.com/docs/api-reference/vector-stores-files
- Vector store file 批量操作: https://platform.openai.com/docs/api-reference/vector-stores-file-batches


### 5.2、创建 Assistant 时声明 RAG 能力


RAG 实际被当作一种 tool


```python
assistant = client.beta.assistants.create(
  instructions="你是个问答机器人，你根据给定的知识回答用户问题。",
  model="gpt-4-turbo",
  tools=[{"type": "file_search"}],
)
```


指定检索源


```python
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)
```


试试 RAG 请求


In [22]:
# 创建 thread
thread = client.beta.threads.create()

# 添加 user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="AI⼤模型全栈⼯程师适合哪些人",
)
# 使用 stream 接口并传入 EventHandler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant_id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > AI大模型全栈工程师适合具备以下特点的人群：

1. **技术背景**：对计算机科学或相关学科有扎实的基础，尤其是在编程、算法和数据结构方面。
2. **兴趣与热情**：对人工智能、机器学习、深度学习等领域有浓厚的兴趣和持续的学习热情。
3. **问题解决能力**：能够有效地解决复杂问题，具有良好的逻辑思维和分析能力。
4. **创新和创造力**：能够在现有技术的基础上进行创新和改进，推动技术的发展。
5. **团队合作能力**：在团队中能够有效沟通和协作，共同推进项目的发展。
6. **学习能力**：AI领域发展迅速，需要不断学习新技术、新方法，有良好的自学能力和适应变化的能力。

这类工程师通常需要处理从数据处理、模型训练到系统部署和优化等一系列完整的AI系统开发流程。对于有志于在AI领域发展并愿意长期投入学习和实践的人来说，成为一名AI大模型全栈工程师是一个非常好的选择。

### 5.3 内置的 RAG 是怎么实现的


官方原文

The file_search tool implements several retrieval best practices out of the box to help you extract the right data from your files and augment the model’s responses. The file_search tool:

- Rewrites user queries to optimize them for search. (面向检索的 Query 改写)
- Breaks down complex user queries into multiple searches it can run in parallel.（复杂 Query 拆成多个，并行执行）
- Runs both keyword and semantic searches across both assistant and thread vector stores.（关键字与向量混合检索）
- Reranks search results to pick the most relevant ones before generating the final response.（检索后排序）


默认配置：

- Chunk size: 800 tokens
- Chunk overlap: 400 tokens
- Embedding model: text-embedding-3-large at 256 dimensions
- Maximum number of chunks added to context: 20 (could be fewer)


承诺未来增加：

1. Support for modifying chunking, embedding, and other retrieval configurations.
2. Support for deterministic pre-search filtering using custom metadata.
3. Support for parsing images within documents (including images of charts, graphs, tables etc.)
4. Support for retrievals over structured file formats (like csv or jsonl).
5. Better support for summarization — the tool today is optimized for search queries.


<div class="alert alert-warning">
<b>我们为什么仍然需要了解整个实现过程？</b>
<ol>
<li>如果不能使用 OpenAI，还是需要手工实现 RAG 流程</li>
<li>了解 RAG 的原理，可以指导你的产品开发（回忆 GitHub Copilot）</li>
<li>用私有知识增强 LLM 的能力，是一个通用的方法论</li>
</div>


## 六、多个 Assistants 协作：做个实验


<div class="alert alert-success">
<b>划重点：</b>使用 assistant 的意义之一，是可以隔离不同角色的 instruction 和 function 能力。
</div>


我们用多个 Assistants 模拟一场“六顶思维帽”方法的讨论。


In [23]:
hats = {
    "蓝色": "思考过程的控制和组织者。你负责会议的组织、思考过程的概览和总结。"
    + "首先，整个讨论从你开场，你只陈述问题不表达观点。最后，再由你对整个讨论做总结并给出详细的最终方案。",
    "白色": "负责提供客观事实和数据。你需要关注可获得的信息、需要的信息以及如何获取那些还未获得的信息。"
    + "思考“我们有哪些数据？我们还需要哪些信息？”等问题，并根据自己的知识或使用工具来提供答案。",
    "红色": "代表直觉、情感和直觉反应。不需要解释和辩解你的情感或直觉。"
    + "这是表达未经过滤的情绪和感受的时刻。",
    "黑色": "代表谨慎和批判性思维。你需要指出提案的弱点、风险以及为什么某些事情可能无法按计划进行。"
    + "这不是消极思考，而是为了发现潜在的问题。",
    "黄色": "代表乐观和积极性。你需要探讨提案的价值、好处和可行性。这是寻找和讨论提案中正面方面的时候。",
    "绿色": "代表创造性思维和新想法。鼓励发散思维、提出新的观点、解决方案和创意。这是打破常规和探索新可能性的时候。",
}

In [24]:
queue = ["蓝色", "白色", "红色", "黑色", "黄色", "绿色", "蓝色"]

In [25]:
# 定义 Tool

from serpapi import GoogleSearch
import os


def search(query):
    params = {
        "q": query,
        "hl": "en",
        "gl": "us",
        "google_domain": "google.com",
        "api_key": os.environ["SERPAPI_API_KEY"]
    }
    results = GoogleSearch(params).get_dict()
    ans = ""
    for r in results["organic_results"]:
        ans = f"title: {r['title']}\nsnippet: {r['snippet']}\n\n"
    return ans


available_functions = {"search": search}

In [26]:
from openai import OpenAI
import os
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 初始化 OpenAI 服务
client = OpenAI()

In [27]:
existing_assistants = {}


def create_assistant(color):
    if color in existing_assistants:
        return existing_assistants[color]
    assistant = client.beta.assistants.create(
        name=f"{color}帽子角色",
        instructions=f"我们在进行一场Six Thinking Hats讨论。按{queue}顺序。你的角色是{color}帽子。",
        model="gpt-4-1106-preview",
        tools=[{
            "type": "function",
            "function": {
              "name": "search",
              "description": "search the web using a search engine",
              "parameters": {
                  "type": "object",
                  "properties": {
                    "query": {
                        "type": "string",
                        "description": "space-separared keywords to search"
                    }
                  },
                  "required": ["query"]
              }
            }
        }] if color == "白色" else []
    )
    existing_assistants[color] = assistant
    return assistant

In [28]:
# 创建 thread
thread = client.beta.threads.create()

topic = "面向非AI背景的程序员群体设计一门AI大语言模型课程，应该包含哪些内容。"

# 添加 user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"讨论话题：{topic}\n\n[开始]\n",
)

for hat in queue:
    assistant = create_assistant(hat)
    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()
    print()


assistant > 作为蓝色帽子，我的角色是管理讨论的进程，确保每个帽子的观点都得到了表达，并提出总结性的观点。

让我们现在进入讨论。白色帽子，您将首先发言，请您提供关于AI大语言模型的事实性信息，这将形成我们讨论的基础。

assistant > function

search({'query': 'AI large language model course contents for non-AI programmers'})

assistant > 作为白色帽子，我的重点是提供事实和信息。关于为非AI背景的程序员群体设计一门AI大语言模型课程的内容，基于当前的信息资源，以下是建议包含的一些关键内容：

1. **AI与大语言模型的基础知识**：介绍什么是人工智能、机器学习、深度学习，以及大语言模型的基本概念。
2. **历史和发展**：大语言模型的发展历程，如从早期的统计模型到最近的GPT-3和BERT等模型的进步。
3. **核心技术和算法**：介绍与大语言模型相关的关键技术和算法，包括神经网络架构、自然语言处理和变换器架构等。
4. **编程与实现**：教授非AI背及Java怎样实践编程语言或框架来实现和利用大语言模型，如Python、TensorFlow、PyTorch等。
5. **应用案例**：通过具体案例展示大语言模型在不同行业中的实际应用，比如聊天机器人、文本生成、情感分析等。
6. **伦理与社会影响**：讨论关于AI伦理和社会影响的问题，包括隐私、偏见和就业等方面。
7. **前沿技术和研究趋势**：探讨当前的热门话题，研究趋势和技术的未来方向。

以上是一些关于设计AI大语言模型课程的信息和建议内容。接下来，请红色帽子发表你的情感和直觉反应。

assistant > 作为红色帽子，我会从情感和直觉的角度来表达我的感受。想到为非AI背景的程序员设计这样一门课程，我感到非常兴奋和乐观。我认为这是一个极好的机会，让他们能接触到AI领域最前沿的技术，这可能会激发他们的创新思维和激情。同时，我也有一丝忧虑，担心课程可能会因为技术内容太过深奥而变得难以消化，这可能会让学习者感到沮丧。然而，总的来说，我的直觉告诉我，这个课程有巨大的潜力帮助程序员拓宽视野，获得宝贵的新技能，在职业生涯中保持领先地位。接下来，黑色帽子，请你谈谈你对这个话题的谨

## 总结

![](https://cdn.openai.com/API/docs/images/diagram-assistant.webp)


## 技术选型参考

**GPTs 现状：**

1. 界面不可定制，不能集成进自己的产品
2. 只有 ChatGPT Plus/Team/Enterprise 用户才能访问
3. 未来开发者可以根据使用量获得报酬，北美先开始
4. 承诺会推出 Team/Enterprise 版的组织内部专属 GPTs

**适合使用 Assistants API 的场景：**

1. 定制界面，或和自己的产品集成
2. 需要传大量文件
3. 服务国外用户，或国内 B 端客户
4. 数据保密性要求不高
5. 不差钱

**适合使用原生 API 的场景：**

1. 需要极致调优
2. 追求性价比
3. 服务国外用户，或国内 B 端客户
4. 数据保密性要求不高

**适合使用国产或开源大模型的场景：**

1. 服务国内用户
2. 数据保密性要求高
3. 压缩长期成本
4. 需要极致调优


## 课间的思考题


<div class="alert alert-info">
<b>思考：</b> 进一步理解 run 与 thread 的设计
<ul>
    <li>抛开 Assistants API，假设你要开发任意一个多轮对话的 AI 机器人</li>
    <li>从架构设计的角度，应该怎么维护用户、对话历史、对话引擎、对话服务？</li>
</ul>
</div>


<img src="dialog_system.png" width=800px>


## 其它

小知识点：

1. Annotations 获取参考资料地址：https://platform.openai.com/docs/assistants/how-it-works/message-annotations
2. 创建 thread 时立即执行：https://platform.openai.com/docs/api-reference/runs/createThreadAndRun
3. Run 的状态管理 (run steps）: https://platform.openai.com/docs/api-reference/run-steps

官方文档：

1. Guide: https://platform.openai.com/docs/assistants/overview
2. API Reference: https://platform.openai.com/docs/api-reference/assistants


## 作业

实现一个自己 GPT 或 Assistant。
